In [1]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/diabetes (1).csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X = df.drop("Outcome",axis=1)
Y=df["Outcome"]

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [8]:
import tensorflow as tf
from keras.layers import Dense,Dropout
from keras.models import Sequential

In [41]:
def model_build(hp):
  model = Sequential()
  count=0
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    if count==0:
      model.add(Dense(hp.Int("units"+str(i),min_value=8,max_value=128),
                      activation=hp.Choice("activation"+str(i),["relu","sigmoid","tanh"]),
                      input_dim=8
                      )
      )
    else:
      model.add(Dense(hp.Int("units"+str(i),min_value=8,max_value=128),
                      activation=hp.Choice("activation"+str(i),["relu","sigmoid","tanh"]),
                      input_dim=8
                      )
      )
    count+=1
  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer=hp.Choice("optimizer",["adam","rmsprop","sgd","adagrad"]),loss="binary_crossentropy",metrics=["accuracy"])
  return model

In [42]:
import kerastuner as kt

In [43]:
# pip install -U keras-tuner

In [44]:
tuner = kt.RandomSearch(model_build,objective="val_accuracy",max_trials=3,directory="mydir2",project_name="kerastuning2")

In [45]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.649350643157959

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 12s


In [46]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1, 'units0': 86, 'activation0': 'tanh', 'optimizer': 'sgd'}

In [47]:
model = tuner.get_best_models(num_models=1)[0]

In [50]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [52]:
model.fit(x_train,y_train,epochs=40,initial_epoch=6,callbacks=[callback],validation_data=(x_test,y_test))

Epoch 7/40
20/20 [==============================] - 0s 6ms/step - loss: 0.4493 - accuracy: 0.7801 - val_loss: 0.4582 - val_accuracy: 0.7792
Epoch 8/40
20/20 [==============================] - 0s 5ms/step - loss: 0.4494 - accuracy: 0.7818 - val_loss: 0.4580 - val_accuracy: 0.7792
Epoch 9/40
20/20 [==============================] - 0s 5ms/step - loss: 0.4493 - accuracy: 0.7801 - val_loss: 0.4580 - val_accuracy: 0.7792


In [54]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
class1=clf.fit(x_train,y_train)

In [55]:
y_pred = class1.predict(x_test)

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8051948051948052

In [58]:
from xgboost import XGBClassifier

In [62]:
clf2=XGBClassifier()
class2=clf2.fit(x_train,y_train)
y_pred2 = class2.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

0.7532467532467533